<a href="https://colab.research.google.com/github/Kishor-Alagappan/house_price_prediction/blob/main/house_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

DATA INGESTION

In [6]:
# Read the CSV file named "train.csv" into a Pandas dataframe and assign it to the variable 'train_df'
train_df = pd.read_csv('/content/train.csv')

# Read the CSV file named "test.csv" into a Pandas dataframe and assign it to the variable 'test_df'
test_df = pd.read_csv('/content/test.csv')

In [7]:
# Display the first 10 rows of the dataframe to provide an initial view of the data
train_df.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,1,2008,WD,Normal,118000


DATA CLEANING

In [8]:
# Check and print the size of the dataframe before removing duplicates
print("Size Before Removing Duplicates", train_df.shape)

# Calculate and print the total number of duplicates in the dataframe
total_duplicates = train_df.duplicated().sum()
print('Total Number of Duplicates', total_duplicates)

# Remove duplicate rows from the dataframe and update 'df' with the result
train_df = train_df.drop_duplicates()

# Print the size of the dataframe after removing duplicates
print("Size After Removing Duplicates", train_df.shape)

Size Before Removing Duplicates (1460, 81)
Total Number of Duplicates 0
Size After Removing Duplicates (1460, 81)


In [9]:
missing_values = train_df.isnull().sum().sort_values(ascending = False)
missing_percent = (train_df.isnull().sum() / train_df.isnull().count()).sort_values(ascending = False)

missing_data = pd.concat([missing_values, missing_percent], axis = 1, keys=['Count','Percentage'])
missing_data.head(25)

,Count,Percentage
PoolQC,1453,0.995205
MiscFeature,1406,0.963014
Alley,1369,0.937671
Fence,1179,0.807534
FireplaceQu,690,0.472603
LotFrontage,259,0.177397
GarageYrBlt,81,0.055479
GarageCond,81,0.055479
GarageType,81,0.055479
GarageFinish,81,0.055479
